In [1]:
%cd ..
%env TOKENIZERS_PARALLELISM=false

d:\Repositories\NLP\AI-CUP-2022-Fall-NLP
env: TOKENIZERS_PARALLELISM=false


In [2]:
import torch
from src import DataLoader, PredictionDatasetForSiamese, SiameseSpanPredictionModel
from src.scoring import compute_score

In [27]:
device = torch.device('cuda')
model_path = 'weights/v1-e0+.pt'
dataset_path = 'data/splitted/test.csv'
batch_size = 1
top_k = 3
max_tokens = 48

In [13]:
model = SiameseSpanPredictionModel()
model.load_state_dict(torch.load(model_path))
model = model.requires_grad_(False).to(device)

In [28]:
from tqdm.auto import tqdm

dataset = PredictionDatasetForSiamese(dataset_path, model.tokenizer)
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, pin_memory=True)

answers = []
for xs, q, r, s, q_state, r_state in tqdm(dataloader):       
    q.__setstate__(q_state)
    r.__setstate__(r_state)
    
    q = q.to(device)
    r = r.to(device)
    s = s.to(device)

    q_special_tokens_mask = q.pop('special_tokens_mask')
    r_special_tokens_mask = r.pop('special_tokens_mask')

    q_valid_mask: torch.Tensor = q.attention_mask.bool()
    q_valid_mask &= ~q_special_tokens_mask.bool()
    q_valid_mask[:, 0] = True

    r_valid_mask: torch.Tensor = r.attention_mask.bool()
    r_valid_mask &= ~r_special_tokens_mask.bool()
    r_valid_mask[:, 0] = True

    preds = model(q, r, s)
    ans = model.decode_answers(xs, q, r, q_valid_mask, r_valid_mask, preds, top_k, max_tokens)
    answers.extend(ans)

  0%|          | 0/798 [00:12<?, ?it/s]

In [29]:
compute_score(answers)

  0%|          | 0/798 [00:00<?, ?it/s]

0.8653439699888482